In [3]:
import os, sys, requests, re, bs4
from bs4 import BeautifulSoup as bs
from queue import Queue


In [56]:
class Token:
    def __init__(self,cookie):
        self.xyz    = requests.Session()
        self.cookie = {'cookie':cookie}
        app_id      = [
            '1348564698517390|007c0a9101b9e1c8ffab727666805038', #--> Meta Portal
            '1174099472704185|0722a7d5b5a4ac06b11450f7114eb2e9', #--> Messenger Kids for iOS
            '155327495133707|a151725bc9b8808a800f4c891505e558' , #--> Facebook Watch for Oculus
            '1331685116912965|e334a1eca4d4ea9ac0c0132a31730663', #--> Facebook Watch for Xbox
            '867777633323150|446fdcd4a3704f64e5f6e5fd12d35d01' , #--> Facebook Watch for Amazon Fire TV
            '437340816620806|04a36c2558cde98e185d7f4f701e4d94' , #--> Facebook Watch for Android TV
            '661587963994814|ffe07cc864fd1dc8fe386229dcb7a05e' , #--> Facebook Watch for Apple TV
            '1692575501067666|3168904bd42ebb12bf981327de99179f', #--> Facebook Watch for Samsung TV
            ]
        for x in app_id:
            self.get_kode(x)
        # self.accessToken = self.get_kode(app_id[0])
    def get_kode(self,act):
        data = {
            'access_token': act,
            'scope': ''}
        req  = self.xyz.post('https://graph.facebook.com/v16.0/device/login/',data=data).json()
        try:
            cd, ucd = req['code'], req['user_code']
            url = 'https://graph.facebook.com/v16.0/device/login_status?method=post&code=%s&access_token=%s'%(cd,act)
            self.verify1(ucd,act)
            access_token = self.get_token(url)
            # return access_token
            print(access_token)
        except Exception as e:

            print('Login Gagal --> %s'%(act))
    def verify1(self,ucd,act):
        req = bs(self.xyz.get('https://mbasic.facebook.com/device',cookies=self.cookie).content,'html.parser')
        raq = req.find('form',{'method':'post'})
        dat = {
            'jazoest'   : re.search('name="jazoest" type="hidden" value="(.*?)"',str(raq)).group(1),
            'fb_dtsg'   : re.search('name="fb_dtsg" type="hidden" value="(.*?)"',str(req)).group(1),
            'qr'        : '0',
            'user_code' : ucd}
        rel = 'https://mbasic.facebook.com' + raq['action']
        pos = bs(self.xyz.post(rel,data=dat,cookies=self.cookie).content,'html.parser')
        self.verify2(pos,ucd,act)
    def verify2(self,req,ucd,act):
        dat = {}
        raq = req.find('form',{'method':'post'})
        for x in raq('input',{'value':True}):
            try:
                if x['name'] == '__CANCEL__' :
                    pass
                else:
                    dat.update({x['name']:x['value']})
            except Exception as e:
                pass
        rel = 'https://mbasic.facebook.com' + raq['action']
        pos = bs(self.xyz.post(rel,data=dat,cookie=self.cookie).content,'html.parser')
        if 'Sukses!' in str(pos):
            pass
        else:
            print('Login Gagal --> %s'%(act))
    def get_token(self,url):
        req = self.xyz.get(url,cookies=self.cookie).json()
        access_token = req['access_token']
        return access_token

In [57]:
cookies = "sb=ttpvZI7d9xfsOc9zFyV0vW4l; datr=vqKBZLbu4PGRjrXWlkKH0_w4; c_user=100013323961498; m_page_voice=100013323961498; usida=eyJ2ZXIiOjEsImlkIjoiQXJ3Ym1vYTF4eW92OXMiLCJ0aW1lIjoxNjg2ODc3MDY2fQ%3D%3D; xs=47%3APwPMsmfVxU3_fA%3A2%3A1686238466%3A-1%3A6253%3A%3AAcWnLDRe0bruqIL4KJnGH4tRHKjpEZL_GubSuATA8Pg; fr=06ddF7aP3jU6AJqdP.AWWi7FcUWXkaieCZLFYkSuqpGYU.Bki7pA.do.AAA.0.0.Bki7pA.AWUCP18MHLM; presence=C%7B%22lm3%22%3A%22u.100010013526023%22%2C%22t3%22%3A%5B%5D%2C%22utc3%22%3A1686878817073%2C%22v%22%3A1%7D; wd=852x947"

pre_accessToken = Token(cookies)
accessToken = pre_accessToken.get_kode('1348564698517390|007c0a9101b9e1c8ffab727666805038')
accessToken

Login Gagal --> 1348564698517390|007c0a9101b9e1c8ffab727666805038
Login Gagal --> 1174099472704185|0722a7d5b5a4ac06b11450f7114eb2e9
Login Gagal --> 155327495133707|a151725bc9b8808a800f4c891505e558
Login Gagal --> 1331685116912965|e334a1eca4d4ea9ac0c0132a31730663
Login Gagal --> 867777633323150|446fdcd4a3704f64e5f6e5fd12d35d01
Login Gagal --> 437340816620806|04a36c2558cde98e185d7f4f701e4d94
Login Gagal --> 661587963994814|ffe07cc864fd1dc8fe386229dcb7a05e
Login Gagal --> 1692575501067666|3168904bd42ebb12bf981327de99179f
Login Gagal --> 1348564698517390|007c0a9101b9e1c8ffab727666805038


In [46]:
def getStoryIDFromPostID(cookie, post_id):
  cookie_raw = {'cookie':cookie}
  with requests.Session() as ses:
    response = ses.get(f'https://www.facebook.com/{post_id}', cookies=cookie_raw, timeout=5)
    print(response.url)
    story_id_match = re.search(r'"storyID":"([^"]+)"', response.text)
    if story_id_match:
        story_id = story_id_match.group(1)
        print(story_id)
    else:
        print("Không tìm thấy storyID trong đoạn mã.")

In [47]:
def removePost(cookies, storyID):
  cookies = {'cookie':cookies}
  headers = {
    'authority': 'www.facebook.com',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    'content-type': 'application/x-www-form-urlencoded',
    'origin': 'https://www.facebook.com',
    'referer': 'https://www.facebook.com',
    'sec-ch-prefers-color-scheme': 'light',
    'sec-ch-ua': '"Chromium";v="103", ".Not/A)Brand";v="99"',
    'sec-ch-ua-full-version-list': '"Chromium";v="103.0.5060.134", ".Not/A)Brand";v="99.0.0.0"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-ch-ua-platform-version': '"10.0.0"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.134 Safari/537.36',
    'viewport-width': '1132',
    'x-asbd-id': '129477',
    'x-fb-friendly-name': 'useCometTrashPostMutation',
    'x-fb-lsd': 'paVhHmefIHy5g8EsbMvA1c',
  }

  data = {
      'av': '100013323961498',
      # '__user': '100013323961498',
      '__a': '1',
      '__comet_req': '15',
      'fb_dtsg': 'NAcNjlCNYEycuxgvM9Y_oIQiefrZUZs_Ova9c0kad2JfZnf-8phwCZw:47:1686238466',
      # 'jazoest': '25358',
      # 'lsd': 'paVhHmefIHy5g8EsbMvA1c',
      'fb_api_caller_class': 'RelayModern',
      'fb_api_req_friendly_name': 'useCometTrashPostMutation',
      'variables': '{"input":{"story_id":"'+storyID+'","story_location":"TIMELINE","actor_id":"100013323961498","client_mutation_id":"1"}}',
      'server_timestamps': 'true',
      'doc_id': '7003056599769350',
  }
  with requests.Session() as ses:
    response = ses.post('https://www.facebook.com/api/graphql/', cookies=cookies, headers=headers, data=data)
    print(response.text)

In [48]:
def getPosts(list_ids_queue):

  cookies = {
      'sb': 'ttpvZI7d9xfsOc9zFyV0vW4l',
      'datr': 'vqKBZLbu4PGRjrXWlkKH0_w4',
      'c_user': '100013323961498',
      'm_page_voice': '100013323961498',
      'xs': '47%3APwPMsmfVxU3_fA%3A2%3A1686238466%3A-1%3A6253%3A%3AAcXK5shdIp5NAJ-j84UxQQ-L-DR-6Gs1lh48clCQX9o',
      'wd': '1920x947',
      'usida': 'eyJ2ZXIiOjEsImlkIjoiQXJ3OXE4OTEweXo4NDYiLCJ0aW1lIjoxNjg2NzkwMTQ4fQ%3D%3D',
      'fr': '0RjbPFmjfeRYwWKQa.AWXISH9zucf1vGYeFd57-OnpMo4.Bkil3B.do.AAA.0.0.BkimAG.AWV-yg01pKk',
      'presence': 'C%7B%22t3%22%3A%5B%7B%22i%22%3A%22u.100018318542484%22%7D%5D%2C%22utc3%22%3A1686790428117%2C%22v%22%3A1%7D',
  }

  headers = {
      'authority': 'graph.facebook.com',
      'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
      'accept-language': 'en-US,en;q=0.9,vi;q=0.8',
      'cache-control': 'max-age=0',
      'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
      'sec-ch-ua-mobile': '?0',
      'sec-ch-ua-platform': '"Windows"',
      'sec-fetch-dest': 'document',
      'sec-fetch-mode': 'navigate',
      'sec-fetch-site': 'none',
      'sec-fetch-user': '?1',
      'upgrade-insecure-requests': '1',
      'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
  }

  params = {
      'limit': '100',
      'access_token': 'EAAGPwr8GD2cBAL6XylMTxaLGKB2e1ZCYyYtppctbTYpwQOTuyGeTfXbf3v12deSyt8tyhh6zZCftC28bEZCY7OZByO3VvKtthHP0IYzK35oYbCL2pA0TiZCrZBD1W5JrU21d06JZA3MxdIybWAGUGGD2w0EpiZBZBTbGSG0lkHSzl7ZCbJw4zK8u2R0cEfsHsxI4PQjkDAJAcCSwZDZD',
  }
  response = requests.get('https://graph.facebook.com/v16.0/me/feed', params=params, cookies=cookies, headers=headers)
  try:
    resp = response.json()
    pre_list = list(map(lambda data : data['id'].split("_")[1], resp['data']))
    _ = list(map(list_ids_queue.put,pre_list))
    while True:
      if(resp['paging'] and resp['paging']['next']):
        response = requests.get(resp['paging']['next'], cookies=cookies, headers=headers)
        resp = response.json()
        pre_list = list(map(lambda data : data['id'].split("_")[1], resp['data']))
        _ = list(map(list_ids_queue.put,pre_list))
      else:
        break
  except Exception as e :
    print(e)

In [49]:
list_ids_queue = Queue()
getPosts(list_ids_queue)

'paging'


In [50]:
list_ids_queue.qsize()
list(list_ids_queue.queue)

['1765347210586061',
 '1742533729534076',
 '1717361968717919',
 '1568357650285019',
 '1690403278080455',
 '1675742806213169',
 '1623346834786100',
 '1616111178842999',
 '1615360672251383',
 '1607966739657443',
 '5939258896119298',
 '1590579408062843',
 '1590041154783335',
 '1585231438597640',
 '1567675870353197',
 '1559436217843829',
 '1554188695035248',
 '1537124973408287',
 '1534360117018106',
 '1529295697524548',
 '1528136700973781',
 '1524036201383831',
 '1523052414815543',
 '1522157778238340',
 '1517002262087225',
 '1515453888908729',
 '1511555472631904',
 '1509895466131238',
 '1509803436140441',
 '1509247799529338',
 '1509245549529563',
 '1507266893060762',
 '1506640829790035',
 '1504964469957671',
 '1504864173301034',
 '1503542456766539',
 '1502457946874990',
 '1502027066918078',
 '1493627787758006',
 '1485966105190841',
 '1482708638849921',
 '1480183662435752',
 '1477597082694410',
 '1472430776544374',
 '1910260672496120',
 '1464210020699783',
 '1452829561837829',
 '14523291852

In [53]:
list_ids_queue.qsize()

167

In [51]:
def getPostIDFromUrl(cookies = "None", urls = "None"):
  cookies = {
      'sb': 'ttpvZI7d9xfsOc9zFyV0vW4l',
      'datr': 'vqKBZLbu4PGRjrXWlkKH0_w4',
      'c_user': '100013323961498',
      'm_page_voice': '100013323961498',
      'xs': '47%3APwPMsmfVxU3_fA%3A2%3A1686238466%3A-1%3A6253%3A%3AAcXrNMSiiyg7QWNGeVbjMY8-moxrIsCiSC4yKwOKjgI',
      'fr': '0kouh4jJbBhbiXEDl.AWXB1jjJBwipkjkcJ0BbfQTfKp8.Bki6iC.do.AAA.0.0.Bki6iC.AWXcgHUmumg',
      'presence': 'C%7B%22t3%22%3A%5B%5D%2C%22utc3%22%3A1686876576240%2C%22v%22%3A1%7D',
      'wd': '852x947',
  }

  headers = {
      'authority': 'www.facebook.com',
      'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
      'accept-language': 'en-US,en;q=0.9,vi;q=0.8',
      'cache-control': 'max-age=0',
      'sec-ch-prefers-color-scheme': 'light',
      'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
      'sec-ch-ua-full-version-list': '"Not.A/Brand";v="8.0.0.0", "Chromium";v="114.0.5735.134", "Google Chrome";v="114.0.5735.134"',
      'sec-ch-ua-mobile': '?0',
      'sec-ch-ua-platform': '"Windows"',
      'sec-ch-ua-platform-version': '"10.0.0"',
      'sec-fetch-dest': 'document',
      'sec-fetch-mode': 'navigate',
      'sec-fetch-site': 'same-origin',
      'sec-fetch-user': '?1',
      'upgrade-insecure-requests': '1',
      'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
      'viewport-width': '852',
  }

  response = requests.get(
      'https://www.facebook.com/truongnhudatt/posts/pfbid0gL2GJ8BMqcj2FEEtsUaaMy5AFMeN8mEbjhR5h95GLUo15s3nNtXaMvGsKky1jS6Bl',
      cookies=cookies,
      headers=headers,
  )
  post_id_match = re.search(r'"post_id":"(\d+)"', response.text)
  if post_id_match:
      post_id = post_id_match.group(1)
      print(post_id)
  else:
      print("Không tìm thấy post_id trong đoạn mã.")


In [52]:
getPostIDFromUrl()

1742533729534076
